# Interactive masking tool

This notebook show how to use the interactive masking tool.

In [ ]:
%matplotlib widget
from scippneutron import MaskingTool
import plopp as pp
import scipp as sc
import numpy as np

We generate some fake data:

In [ ]:
rng = np.random.default_rng(seed=4321)
position = 10.0 * rng.standard_normal(size=[50_000, 2])
values = np.linalg.norm(position, axis=1)
da = sc.DataArray(
    data=sc.array(dims=["row"], values=values, unit="K"),
    coords={
        "x": sc.array(dims=["row"], unit="m", values=position[:, 0]),
        "y": sc.array(dims=["row"], unit="m", values=position[:, 1]),
    },
).hist(y=300, x=300)
da

## Two-dimensional masks

We then load it into the `MaskingTool`.

**Instructions:**

- Use the buttons in the top bar to add masks to the data
- Left-click to add a new shape, and left-click again to persist the shape
- Left-click a vertex to edit a shape
- Right-click and hold to drag a shape
- Middle-click (or Ctrl + left-click) to delete a shape
- Save the masks to a file when the "Save" button is clicked

In [ ]:
masking_tool = MaskingTool(da, norm="log")

In [ ]:
r, v, h = masking_tool.controls

r.value = True
r._tool.click(-20, 5)
r._tool.click(1, 30)

v.value = True
v._tool.click(7, 0)
v._tool.click(15, 0)
v._tool.click(20, 0)
v._tool.click(30, 0)

h.value = True
h._tool.click(0, -20)
h._tool.click(0, -10)

masking_tool.filename.value = "masks-2d.json"

def update(fig):
    from ipywidgets import HBox

    fig.children = [
        fig.top_bar,
        HBox([fig.left_bar, fig.canvas.to_image(), fig.right_bar]),
        fig.bottom_bar,
    ]

update(masking_tool.fig)

In [ ]:
masking_tool

You can either save the masks to a file by filling in the text field and clicking the "Save" button,
or get the masks as a `dict` by calling:

In [ ]:
masking_tool.get_masks()

## One-dimensional masks

The tool also works with one-dimensional data.
In this case, only the vertical span tool is active.

In [ ]:
masking_tool = MaskingTool(da.sum("y"), norm="log")

In [ ]:
v = masking_tool.controls[1]
v.value = True
v._tool.click(-40, 150)
v._tool.click(-20, 150)
v._tool.click(20, 150)
v._tool.click(40, 150)

masking_tool.filename.value = "masks-1d.json"

update(masking_tool.fig)

In [ ]:
masking_tool

## Using the masks from the tool

We will now show how we can connect the output of the masking tool to further computation,
applying the masks and processing the data further.

We first begin by setting up the same tool as above (in 2D).

In [ ]:
masking_tool = MaskingTool(da, norm="log")

We then create a node that will listen to the changes in the masking tool and apply the masks onto the data before summing along the vertical dimension.

In [ ]:
def apply_masks(da, trigger_node):
    # Get the current masks
    masks = masking_tool.get_masks()
    # Make a shallow copy and add the masks
    out = da.copy(deep=False)
    for name in masks:
        cond = sc.ones(sizes=da.sizes, dtype=bool)
        for dim, bounds in masks[name]["bounds"].items():
            mids = sc.midpoints(da.coords[dim])
            m = (
                mids > sc.scalar(bounds["min"]["value"], unit=bounds["min"]["unit"])
            ) & (mids < sc.scalar(bounds["max"]["value"], unit=bounds["max"]["unit"]))
            cond = cond & m
        out.masks[name] = cond
    # Reduce data and return
    return out.sum("y")


apply = pp.Node(apply_masks, da=da, trigger_node=masking_tool.masking_node)

fig = pp.plot(apply, norm="log")

We now show the two figures, and changes in the tool should update the reduced result.
Adding a rectangle lowers the signal in the same x range but does not bring it to zero.
Adding a vertical span zeros the data.

In [ ]:
r, v, h = masking_tool.controls

r.value = True
r._tool.click(-20, -5)
r._tool.click(-5, 20)

v.value = True
v._tool.click(10, 0)
v._tool.click(18, 0)

update(masking_tool.fig)
update(fig)

In [ ]:
pp.widgets.Box([masking_tool.fig, fig])